### Imports

In [1]:
%pylab inline
import argparse
from nipype.interfaces import fsl
from nipype.algorithms.modelgen import SpecifyModel
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.pipeline.engine import Workflow, Node
from os.path import join
from utils.event_utils import move_EVs
import sys

Populating the interactive namespace from numpy and matplotlib
180427-09:41:13,671 duecredit ERROR:
	 Failed to import duecredit due to No module named 'duecredit'


### Parse Arguments
These are not needed for the jupyter notebook, but are used after conversion to a script for production

- conversion command:
  - jupyter nbconvert --to script --execute task_analysis.ipynb

In [2]:
parser = argparse.ArgumentParser(description='Example BIDS App entrypoint script.')
parser.add_argument('-derivatives_dir', default='/output')
parser.add_argument('-data_dir', default='/data')
parser.add_argument('--participant_label',nargs="+")
parser.add_argument('--events_dir', default=None)
parser.add_argument('--tasks', nargs="+")
parser.add_argument('--use_events', action='store_false')
parser.add_argument('--ignore_rt', action='store_true')
parser.add_argument('--cleanup', action='store_true')
parser.add_argument('--overwrite_event', action='store_true')
if '-output_dir' in sys.argv:
    args = parser.parse_args()
else:
    args = parser.parse_args([])

In [ ]:
args.derivatives_dir = '/home/ian/tmp/fmri/derivatives/'
args.data_dir = '/home/ian/tmp/fmri/data/'

### Initial Setup

In [3]:
# list of subject identifiers
subject_list = args.participant_label
# list of task identifiers
if args.tasks:
    task_list = args.tasks
else:
  task_list = ['ANT', 'CCTHot', 'discountFix',
               'DPX', 'motorSelectiveStop',
               'stopSignal', 'stroop', 'surveyMedley',
               'twoByTwo', 'WATT3']

regress_rt = not args.ignore_rt
use_events = args.use_events
#### Experiment Variables
output_dir = args.output_dir
events_dir = args.events_dir
data_dir = args.data_dir
first_level_dir = '1stLevel'
working_dir = 'workingdir'
# TR of functional images
TR = .68

# Set up Nodes

### Define helper functions

In [21]:
# helper function to create bunch
def getsubjectinfo(data_dir, fmriprep_dir, subject_id, task, regress_rt=True): 
    from glob import glob
    from os.path import join
    import pandas as pd
    from nipype.interfaces.base import Bunch
    from utils.event_utils import get_contrasts, parse_EVs, process_confounds
    # strip "sub" from beginning of subject_id if provided
    subject_id = subject_id.replace('sub-','')
    ## Get the Confounds File (output of fmriprep)
    # Read the TSV file and convert to pandas dataframe
    confounds_file = glob(join(fmriprep_dir,
                               'sub-%s' % subject_id,
                               '*', 'func',
                               '*%s*confounds.tsv' % task))[0]
    regressors, regressor_names = process_confounds(confounds_file)
    ## Get the Events File if it exists
    # Read the TSV file and convert to pandas dataframe
    event_file = glob(join(data_dir,
                           'sub-%s' % subject_id,
                           '*', 'func',
                           '*%s*events.tsv' % task))
    if len(event_file)>0:
        event_file = event_file[0]
        events_df = pd.read_csv(event_file,sep = '\t')
        # set up contrasts
        EV_dict = parse_EVs(events_df, task, regress_rt)
        contrast_subjectinfo = Bunch(subject_id=subject_id,
                                      task=task,
                                      conditions=EV_dict['conditions'],
                                      onsets=EV_dict['onsets'],
                                      durations=EV_dict['durations'],
                                      amplitudes=EV_dict['amplitudes'],
                                      tmod=None,
                                      pmod=None,
                                      regressor_names=regressor_names,
                                      regressors=regressors.T.tolist())
        contrasts = get_contrasts(task, regress_rt)
    else:
        contrast_subjectinfo = Bunch()
        contrasts=[]
    # create base_subject info
    base_subjectinfo = Bunch(subject_id=subject_id,
                             task=task,
                             tmod=None,
                             pmod=None,
                             regressor_names=regressor_names,
                             regressors=regressors.T.tolist())

    return contrast_subjectinfo, base_subjectinfo, contrasts # this output will later be returned to infosource

def save_subjectinfo(base_directory, subject_id, task, subjectinfo, contrasts):
    from os import makedirs
    from os.path import join
    import pickle
    task_dir = join(base_directory, subject_id + '_task_' + task)
    makedirs(task_dir, exist_ok=True)
    subjectinfo_path = join(task_dir,'subjectinfo.pkl')
    pickle.dump(subjectinfo, open(subjectinfo_path,'wb'))
    if len(contrasts) > 0:
        contrast_path = join(task_dir,'contrasts.pkl')
        pickle.dump(contrasts, open(contrast_path,'wb'))

View one events file used in subject info

### Specify Input and Output Stream

In [22]:
# Get Subject Info - get subject specific condition information
subjectinfo = Node(Function(input_names=['data_dir', 'fmiriprep_dir',
                                         'subject_id', 'task','regress_rt'],
                               output_names=['contrast_subjectinfo', 
                                             'base_subjectinfo',
                                             'contrasts'],
                               function=getsubjectinfo),
                      name='getsubjectinfo')
subjectinfo.inputs.fmriprep_dir = fmriprep_dir
subjectinfo.inputs.data_dir = data_dir
subjectinfo.inputs.regress_rt = regress_rt

# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id',
                                            'task']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list),
                        ('task', task_list)]
# SelectFiles - to grab the data (alternative to DataGrabber)
templates = {'func': join('*{subject_id}','*','func',
                         '*{task}*MNI*preproc.nii.gz'),
            'mask': join('*{subject_id}','*','func',
                         '*{task}*MNI*brainmask.nii.gz')}
selectfiles = Node(SelectFiles(templates,
                               base_directory = data_dir,
                               sort_filelist=True),
                   name="selectfiles")
# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory = output_dir,
                         container=first_level_dir),
                name="datasink")
# Save python objects that aren't accomodated by datasink nodes
save_subjectinfo = Node(Function(input_names=['base_directory','subject_id',
                                              'task','subjectinfo','contrasts'],
                                 output_names=['output_path'],
                                function=save_subjectinfo),
                       name="savesubjectinfo")
save_subjectinfo.inputs.base_directory = join(output_dir,first_level_dir)

# Use the following DataSink output substitutions
substitutions = [('_subject_id_', ''),
                ('fstat', 'FSTST'),
                ('run0.mat', 'designfile.mat')]
datasink.inputs.substitutions = substitutions

### Model Specification

In [24]:
# mask and blur
masker = Node(fsl.maths.ApplyMask(),name='masker')

# SpecifyModel - Generates FSL-specific Model
modelspec = Node(SpecifyModel(input_units='secs',
                              time_repetition=TR,
                              high_pass_filter_cutoff=80),
                 name="modelspec")
# Level1Design - Creates FSL config file 
level1design = Node(fsl.Level1Design(bases={'dgamma':{'derivs': True}},
                                     interscan_interval=TR,
                                     model_serial_correlations=True),
                        name="level1design")
# FEATmodel generates an FSL design matrix
level1model = Node(fsl.FEATModel(), name="FEATModel")

# FILMGLs
# smooth_autocorr, check default, use FSL default
filmgls = Node(fsl.FILMGLS(), name="GLS")

# Create workflow

### helper functions

In [25]:
def init_wf(name='wf'):
  wf = Workflow(name=name)
  wf.connect([(modelspec, level1design, [('session_info','session_info')]),
              (level1design, level1model, [('ev_files', 'ev_files'),
                                             ('fsf_files','fsf_file')]),
              (level1model, filmgls, [('design_file', 'design_file'),
                                        ('con_file', 'tcon_file'),
                                        ('fcon_file', 'fcon_file')]),
              (level1model, datasink, [('design_file', '@design_file')]),
              (filmgls, datasink, [('copes', '%s.@copes' % name),
                                    ('zstats', '%s.@Z' % name),
                                    ('fstats', '%s.@F' % name),
                                    ('tstats','%s.@T' % name),
                                    ('param_estimates','%s.@param_estimates' % name),
                                    ('residual4d', '%s.@residual4d'),
                                    ('sigmasquareds', '%s.@sigmasquareds' % name)])])

In [26]:
# workflow for calculating contrasts
wf1 = init_wf(name='wf1')
# workflow for only removing nuisance variables
wf2 = init_wf(name='wf2')

In [27]:
# Initiation of the 1st-level analysis workflow
l1analysis = Workflow(name='l1analysis')
l1analysis.base_dir = join(output_dir, working_dir)
# Connect up the 1st-level analysis components
l1analysis.connect([(infosource, selectfiles, [('subject_id', 'subject_id'),
                                               ('task', 'task')]),
                    (infosource, subjectinfo, [('subject_id','subject_id'),
                                                 ('task', 'task')]),
                    (subjectinfo, save_subjectinfo, [('subject_id','subject_id'),
                                                        ('task','tasj'),
                                                        ('subjectinfo','subjectinfo'),
                                                        ('contrasts','contrasts')]),
                    (selectfiles, masker, [('func','in_file'),
                                           ('mask', 'mask_file')])
                    ])

In [20]:
l1analysis.run()

180427-09:42:21,609 workflow INFO:
	 Workflow l1analysis settings: ['check', 'execution', 'logging', 'monitoring']
180427-09:42:21,615 workflow INFO:
	 Running serially.
180427-09:42:21,616 workflow INFO:
	 Executing node l1analysis.selectfiles in dir: /home/ian/tmp/fmri/derivatives/workingdir/l1analysis/_subject_id_s130_task_stroop/selectfiles
180427-09:42:21,620 workflow INFO:
	 Running node "selectfiles" ("nipype.interfaces.io.SelectFiles").
180427-09:42:21,627 workflow INFO:
	 Executing node l1analysis.getsubjectinfo in dir: /home/ian/tmp/fmri/derivatives/workingdir/l1analysis/_subject_id_s130_task_stroop/getsubjectinfo
180427-09:42:21,631 workflow INFO:
	 Running node "getsubjectinfo" ("nipype.interfaces.utility.wrappers.Function").
180427-09:42:21,640 workflow ERROR:
	 Node getsubjectinfo.a0 failed to run on host ian-System-Product-Name.
180427-09:42:21,641 workflow ERROR:
	 Saving crash info to /media/Data/Ian/Experiments/expfactory/Self_Regulation_Ontology_fMRI/fmri_analysis/sc

RuntimeError: Workflow did not execute cleanly. Check log for details

In [ ]:
# add on contrast wf
l1analysis.connect([
                    (subjectinfo, wf1, [('contrast_subjectinfo','modelspec.subject_info')]),
                    (masker, wf1, [('out_file', 'modelspec.functional_runs')]),
                    (subjectinfo, wf1, [('contrasts','level1design.contrasts')]),
                    (masker, wf1, [('out_file','GLS.in_file')])
                    ])
# add on residual wf
l1analysis.connect([
                    (subjectinfo, wf2, [('base_subjectinfo','modelspec.subject_info')]),
                    (masker, wf2, [('out_file', 'modelspec.functional_runs')]),
                    (masker, wf2, [('out_file','GLS.in_file')])
                    ])

Run workflow

In [ ]:
l1analysis.run('MultiProc', plugin_args={'n_procs': 4})

### Run the Workflow


In [ ]:
l1analysis.run('MultiProc', plugin_args={'n_procs': 4})

### Visualize Workflow

### Visualize Design Matrix

### Visualize Results